Step 1 - Scraping

In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup
import requests
import shutil
import pymongo
from splinter import Browser
import tweepy
import time
import pandas as pd

In [2]:
# Initiate browser helper function
def init_browser():
    exec_path = {'executable_path': '/usr/local/bin/chromedriver'}
    return Browser('chrome', **exec_path, headless=True)

In [3]:
#List of URLs to explore and scrape
url1 = 'https://mars.nasa.gov/news/'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url3 = 'https://twitter.com/marswxreport?lang=en'
url4 = 'http://space-facts.com/mars/'
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

NASA Mars News

In [4]:
# Visit Mars News URL (url1)
browser = init_browser()
browser.visit(url1)

In [6]:
# Retrieve page with the requests module
html = requests.get(url1)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')

# Get title & paragraph text
news_title_1 = soup.find('div', 'content_title', 'a').text
news_p_1 = soup.find('div', 'rollover_description_inner').text

In [7]:
news_title = news_title_1.strip()
news_p = news_p_1.strip()

In [8]:
print(news_title)
print(news_p)

Opportunity Hunkers Down During Dust Storm
It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home.


JPL Mars Space Images

In [ ]:
# Visit Mars Featured Image (url2)
browser = init_browser()
browser.visit(url2)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand = browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)

# Create BeautifulSoup object; parse with 'html.parser'
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

# Get the featured image url
img_relative = jpl_soup.find('img', class_='fancybox-image')['src']
featured_image_url = f'https://www.jpl.nasa.gov{img_relative}'
print(featured_image_url)

In [ ]:
# Download and save the image from 'feautured_image'url
response = requests.get(featured_image_url,stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

In [ ]:
# Display image 
from IPython.display import Image
Image(url='img.jpg')

Mars Weather

In [ ]:
# Visit Mars Weather URL (url3)
browser = init_browser()
browser.visit(url3)

In [ ]:
# Get weather using BeautifulSoup
html_weather = browser.html
soup = BeautifulSoup(html_weather, "html.parser")
for text in browser.find_by_css('.tweet-text'):
    if text.text.partition(' ')[0] == 'Sol':
        mars_weather = text.text
        break
print(mars_weather)

Mars Facts

In [ ]:
# Visit Mars Facts URL (url4)
browser = init_browser()
browser.visit(url4)
time.sleep(1)

# Get mars facts using BeautifulSoup
mars_facts_html = browser.html
mars_facts_soup = BeautifulSoup(mars_facts_html, 'html.parser')

fact_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

In [ ]:
# Visit Mars Hemispheres URL (url5) & Loop through the four tags

mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = BeautifulSoup(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)

Step 2 - MongoDB and Flask Application

Convert Jupyter notebook into a Python script called scrape_mars.py